# Alpaca Paper Trading Using FinRL

Disclaimer: This is intended as a RL educational material and by no means is a recommendation to trade with real money. 

This notebook provide a working version of [Alpaca Paper Trading notebook](https://github.com/AI4Finance-Foundation/FinRL/blob/master/examples/FinRL_PaperTrading_Demo.ipynb) from [FinRL](https://github.com/AI4Finance-Foundation/FinRL/tree/master).  

In [ ]:
%load_ext autoreload
%autoreload 2

## [Install FinRL](https://finrl.readthedocs.io/en/latest/start/installation.html)

In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

## Set Alpaca API Keys

Register to [Alpaca](https://alpaca.markets/) an account and generate and copy your API Key ID & Secret ID below.

In [1]:
API_KEY = ""
API_SECRET = ""
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'

## Imports

In [ ]:
import datetime
from pandas.tseries.offsets import BDay  # business day

from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.meta.data_processor import DataProcessor
from finrl.meta.paper_trading.alpaca import PaperTradingAlpaca

from paper_trading.common import train, test

## Data

We fetch some data and perform data cleaning and feature engineering. 

### DOW 30 Symbols

In [ ]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)

In [ ]:
print(ticker_list)

In [ ]:
print(INDICATORS)

### Fetch Data

In [ ]:
today = datetime.datetime.today()
TEST_END_DATE = (today - BDay(1)).to_pydatetime().date()
TEST_START_DATE = (TEST_END_DATE - BDay(1)).to_pydatetime().date()
TRAIN_END_DATE = (TEST_START_DATE - BDay(1)).to_pydatetime().date()
TRAIN_START_DATE = (TRAIN_END_DATE - BDay(5)).to_pydatetime().date()
TRAINFULL_START_DATE = TRAIN_START_DATE
TRAINFULL_END_DATE = TEST_END_DATE

In [ ]:
TRAIN_START_DATE = str(TRAIN_START_DATE)
TRAIN_END_DATE = str(TRAIN_END_DATE)
TEST_START_DATE = str(TEST_START_DATE)
TEST_END_DATE = str(TEST_END_DATE)
TRAINFULL_START_DATE = str(TRAINFULL_START_DATE)
TRAINFULL_END_DATE = str(TRAINFULL_END_DATE)

In [ ]:
print("TRAIN_START_DATE: ", TRAIN_START_DATE)
print("TRAIN_END_DATE: ", TRAIN_END_DATE)
print("TEST_START_DATE: ", TEST_START_DATE)
print("TEST_END_DATE: ", TEST_END_DATE)
print("TRAINFULL_START_DATE: ", TRAINFULL_START_DATE)
print("TRAINFULL_END_DATE: ", TRAINFULL_END_DATE)

In [ ]:
DP = DataProcessor(
    data_source = 'alpaca',
    API_KEY = API_KEY, 
    API_SECRET = API_SECRET, 
    API_BASE_URL = API_BASE_URL
)

In [ ]:
data = DP.download_data(start_date = TEST_START_DATE, 
                       end_date = TEST_END_DATE, 
                       ticker_list = ticker_list, 
                       time_interval= '1Min')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data['timestamp'].nunique()

### Data Cleaning & Feature Engineering

In [ ]:
data = DP.clean_data(data)
data = DP.add_technical_indicator(data, INDICATORS)
data = DP.add_vix(data)

In [ ]:
data.head()

In [ ]:
data.shape

## Train the Agent

### Train

In [ ]:
ERL_PARAMS = {
    "learning_rate": 3e-6,
    "batch_size": 2048,
    "gamma":  0.985,
    "seed":312,
    "net_dimension":[128, 64], 
    "target_step":5000, 
    "eval_gap":30,
    "eval_times":1
} 
env = StockTradingEnv

In [ ]:
train(start_date = TRAIN_START_DATE, 
      end_date = TRAIN_END_DATE,
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env,
      model_name='ppo',
      if_vix=True, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl', #current_working_dir
      break_step=1e5)

### Test

In [ ]:
account_value_erl = test(start_date = TEST_START_DATE, 
                      end_date = TEST_END_DATE,
                      ticker_list = ticker_list, 
                      data_source = 'alpaca',
                      time_interval= '1Min', 
                      technical_indicator_list= INDICATORS,
                      drl_lib='elegantrl', 
                      env=env, 
                      model_name='ppo',
                      if_vix=True, 
                      API_KEY = API_KEY, 
                      API_SECRET = API_SECRET, 
                      API_BASE_URL = API_BASE_URL,
                      cwd='./papertrading_erl',
                      net_dimension = ERL_PARAMS['net_dimension'])

## Full Training 

After tuning well, retrain on the training and testing sets

In [ ]:
train(start_date = TRAINFULL_START_DATE, 
      end_date = TRAINFULL_END_DATE,
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= '1Min', 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env, 
      model_name='ppo',
      if_vix=True, 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl_retrain',
      break_step=2e5)

## Deploy the Agent

Play around with different parameters e.g., turbulence_thresh, max_stock etc and check the Alpaca web interface. To stop the trading, stop the following cell.   

In [ ]:
# action and state dimensions 
action_dim = len(DOW_30_TICKER)
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

In [ ]:
paper_trading_erl = PaperTradingAlpaca(ticker_list = DOW_30_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './papertrading_erl_retrain', 
                                       net_dim = ERL_PARAMS['net_dimension'], 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = API_KEY, 
                                       API_SECRET = API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS, 
                                       turbulence_thresh=30, 
                                       max_stock=5)
paper_trading_erl.run()